# Capstone proposal by Valentin

# CRM Predictions

# 1 - GCP SQL Queries

# Anonymizing

One of the attention point of the project is working with sensitive data
> WORKING WITH SENSITIVE DATA : 
It is possible to work with sensitive data, e.g. data from your employer. In this case, we first recommend you to anonymize your dataset. For example, you can change the label name of your features or rescale the values before sharing. This step should already take care of most concerns around sensitive data. As a second alternative, you could also prepare a small example dataset (which you manipulate in such a way that it’s not sensitive anymore) that showcases the structure and general information of your dataset.

In my case the following safeguards are in place
- All CustomerIDs are anonymized using SHA256, which has the benefit of anonymizing to the same value in each SQL Query, which is important for JOIN to be possible.
`SHA256(CAST(header.ocdContactMasterId AS STRING)) AS CustomerId`
- All Franchise, ItemIDs are anonymized using SHA256, which makes it impossible which product is contributing. `SHA256(o.eanCode) as lineID`
- All amounts are multiplied by a hidden factor example `header.purchaseAmountTaxIncludedBeforeDiscount * factor as orderAmountBeforeDiscount`
- The context and number of brands selected is unknown `REGEXP_CONTAINS(contextMaster.brand, brands)`

You will see at the end of the next data preparation notebook, that we take an additional measure by taking out a random sample (while keeping certains dimensions proportions intact) out of our total dataset, to add another safeguard and render volumes meaningless

## Contacts

This query retrieves data from contact_master, which contains data deduplicated at the customerID level. It selects distinct values of the following columns:

* __CustomerId__: unique identifier for each contact, obtained by hashing the ocdMasterId column of the contextMaster table as a string using SHA256
* __contactAcquisitionDate__: The date when the contact was acquired.
* __Language__: language preference of the contact.
* __gender__: gender of the contact
* __RecruitmentSource__: source from which the contact was recruited (on the brand website, in a point of Sales, or in a media operation)
* __zipCode__: The postal code of the contact
* __birthday__: The date of birth of the contact.
* __emailDomain__: The domain name of the contact's email address, extracted from the emailAddress column. Restricted to top values, otherwise value is 'other'. Displaying specific email domains, such as company email would pose a data privacy issue, and displaying exhaustive list would be problematic for potential one hot encoding.

Contacts that have been anonymized, are generic, or acquired through the customercare platform are excluded.

### Query

``` sql
SELECT distinct
  SHA256(CAST(contextMaster.ocdMasterId AS STRING)) AS CustomerId,
  DATE(contactAcquisitionDate) AS contactAcquisitionDate,
  CASE 
    WHEN REGEXP_CONTAINS(preferences.language.language, "fr") THEN "fr" 
    WHEN REGEXP_CONTAINS(preferences.language.language, "it") THEN "it" 
    ELSE "de" 
  END AS Language, 
  identityInfo.gender.gender,
  RecruitmentSource, 
  contactInfo.postal.zipCode,
  birthday,
  CASE 
    WHEN SUBSTR(emailAddress, INSTR(emailAddress, '@') + 1) IN ("gmail.com", "hotmail.com", "bluewin.ch", "gmx.ch", "yahoo.com", "icloud.com", "outlook.com", "hotmail.ch", "yahoo.fr", "hotmail.fr", "sunrise.ch", "gmx.net", "yahoo.de", "hispeed.ch", "bluemail.ch") THEN SUBSTR(emailAddress, INSTR(emailAddress, '@') + 1)
    ELSE "other"
  END AS emailDomain,

FROM `emea-c1-dwh-prd.switzerland_all.contact_master`
WHERE 
  REGEXP_CONTAINS(contextMaster.brand, brands)
  AND NOT anonymizedContact
  AND NOT genericContact
  AND NOT cCareNonOptinContact
  AND acquisitionSourcePlatform <> "customercare"

ORDER BY 1

```

## Orders

This SQL query extracts data on customer orders and items purchased. The query uses 5 Common Table Expressions (CTEs).

* First two CTEs, __offline_orders__ and __online_orders__, extract information on offline and online orders respectively, including
    * customer IDs
    * order IDs, order dates, order amounts
    * item-level details such as item IDs, item quantities, and line amounts.
* Third CTE, __dedup_orders__ removes duplicate orders from both offline and online sources, keeping only unique orders per customer and ranking them by order date. Fourth CTE, __OrderNumberDB__, attaches to each order the time in the purchase path of the corresponding customer, or the order rank.

* Fifth CTE, __UniqueEANS__, deduplicates item-level data by grouping them by item IDs and selecting the most recent sales category, subcategory, and franchise values.

Finally, they are combined a table with
* customer IDs
* source channels (offline or online)
* order IDs, order dates, order amounts
* item IDs, item quantities, line amounts, and various item classifications such as category, subcategory, and franchise. 

### Query

``` sql
-- CTE 1: OFFLINE orders with CustomerID, OrderID and ItemID
-- hence duplicated lines for CustomerID and OrderID 
WITH offline_orders AS (
  SELECT distinct
    SHA256(CAST(header.ocdContactMasterId AS STRING)) AS CustomerId,
    -- order level
    'offline' AS sourceChannel,
    SHA256(CAST(context.ocdTicketId AS STRING)) AS OrderId,
    header.ticketDate AS OrderDate,
    header.purchaseAmountTaxIncludedBeforeDiscount * factor as orderAmountBeforeDiscount,
    header.purchaseAmountTaxIncludedAfterDiscount * factor as orderAmount,
    IF(header.purchaseAmountTaxIncludedBeforeDiscount = header.purchaseAmountTaxIncludedAfterDiscount, 0, 1) AS orderPromo,

    -- ean level
    hits.eanCode,
    hits.itemQuantity,
    hits.lineAmountTaxIncludedBeforeDiscount * factor as lineAmountBeforeDiscount,
    hits.lineAmountTaxIncludedAfterDiscount * factor as lineAmount,
    IF(hits.lineAmountTaxIncludedBeforeDiscount = hits.lineAmountTaxIncludedAfterDiscount, 0, 1) AS linePromo


  FROM `emea-c1-dwh-prd.switzerland_all.sales_history`
  CROSS JOIN UNNEST(lines) as hits 
  WHERE validPurchase and REGEXP_CONTAINS(context.brand, brands)
  and date(header.ticketDate) >= startDate
  and date(header.ticketDate) <= endDate
  and hits.lineAmountTaxIncludedBeforeDiscount > 0
),

-- CTE 2: ONLINE orders with CustomerID, OrderID and ItemID
-- hence duplicated lines for CustomerID and OrderID 
online_orders AS (
  SELECT distinct
    SHA256(CAST(header.ocdContactMasterId AS STRING)) AS CustomerId,
    -- order level
    'online' AS sourceChannel,
    SHA256(CAST(context.ocdOrderId AS STRING)) AS OrderId,
    header.orderDate AS OrderDate,
    header.merchandizeTotalAmount.merchandizeTotalAmountTaxIncludedBeforeDiscount * factor as orderAmountBeforeDiscount,
    header.merchandizeTotalAmount.merchandizeTotalAmountTaxIncludedAfterDiscount * factor as orderAmount,
    IF(header.merchandizeTotalAmount.merchandizeTotalAmountTaxIncludedBeforeDiscount = header.merchandizeTotalAmount.merchandizeTotalAmountTaxIncludedAfterDiscount, 0, 1) AS orderPromo,


    -- ean level
    hits.eanCode,
    hits.itemQuantity,
    hits.lineAmountTaxIncludedBeforeDiscount * factor as lineAmountBeforeDiscount,
    hits.lineAmountTaxIncludedAfterDiscount * factor as lineAmount,
    
    IF(hits.lineAmountTaxIncludedBeforeDiscount = hits.lineAmountTaxIncludedAfterDiscount, 0, 1) AS linePromo


  FROM `emea-c1-dwh-prd.switzerland_all.order`
  CROSS JOIN UNNEST(lines) as hits
  WHERE validPurchase and REGEXP_CONTAINS(context.brand, brands)
  and date(header.orderDate) >= startDate
  and date(header.orderDate) <= endDate
  and hits.lineAmountTaxIncludedBeforeDiscount> 0
),

-- CTE 3: Unique OrderID per CustomerID, ranked OrderDate
dedup_orders AS (
  SELECT DISTINCT o.CustomerId, o.OrderId, o.OrderDate
  FROM (
    SELECT * FROM offline_orders
    UNION ALL
    SELECT * FROM online_orders
  ) o
  WHERE o.CustomerId IS NOT NULL
  GROUP BY 1, 2, 3
  ORDER BY 1, 3
),

-- CTE 4: Attach to each Order ID the time in the purchase path of CustomerID
-- OrderRank denotates whether Order was first, second or third order (etc ..)
OrderNumberDB AS (
SELECT DISTINCT CustomerId, OrderId, OrderDate, 
  ROW_NUMBER() OVER (PARTITION BY CustomerId ORDER BY OrderDate ASC) AS OrderRank
FROM dedup_orders 
ORDER BY 1, 3),


-- CTE 5 : Item information deduplicated at itemID level
-- Cleaning of unused or very low frequency item categories / subcategories
UniqueEANS AS (
  SELECT distinct
    eanCode, 
    FIRST_VALUE(category) OVER (PARTITION BY eanCode ORDER BY variantSalesOpenDate DESC) as category,
    FIRST_VALUE(subcategory) OVER (PARTITION BY eanCode ORDER BY variantSalesOpenDate DESC) as subcategory,
    FIRST_VALUE(franchise) OVER (PARTITION BY eanCode ORDER BY variantSalesOpenDate DESC) as franchise,  
  FROM (
    SELECT DISTINCT variantInfo.eanCode,
      variantInfo.variantSalesOpenDate,
      CASE 
        WHEN classification.classificationAxis IN ('Miscellaneous', 'Fragrance', 'MakeUp') THEN 'Skin Care'
        ELSE IFNULL(classification.classificationAxis, "Skin Care")
      END AS category,
      IFNULL(
        CASE 
          WHEN classification.classificationSubAxis IN ('Miscellaneous', 'Miscellaneous Cosmetic', 'Women Fragrance', 'Face Care Caring', 'Lip Makeup') THEN 'Face Care'
          WHEN classification.classificationSubAxis = 'Deodorant' THEN 'Deodorants'
          WHEN classification.classificationSubAxis IN ('Health Hygiene', 'Bath & Shower') THEN 'Soaps'
          WHEN classification.classificationSubAxis IN ('Hair Care', 'Styling', 'Other Hair') THEN 'Hair'
          WHEN classification.classificationSubAxis IN ('Face Care for Men', 'Face Cleansing for Men') THEN 'Men Skin Care'
          WHEN classification.classificationSubAxis = 'Face Care Cleansing' THEN 'Face Cleansing'
          ELSE classification.classificationSubAxis
        END,
        "Face Care"
      ) AS subcategory,
      IFNULL(classification.classificationSubBrand, "OtherBrand") AS franchise
    FROM `emea-c1-dwh-prd.switzerland_all.item`
    WHERE REGEXP_CONTAINS(context.brand, brands)
    AND NOT REGEXP_CONTAINS(variantInfo.eanCode, ',')
  )
)


SELECT o.CustomerId,
       o.sourceChannel,
       o.OrderId,
       o.OrderDate,
       o.orderAmountBeforeDiscount,
       o.orderAmount,
       o.orderPromo,
       a.orderRank,
       SHA256(o.eanCode) as lineID,
       o.lineAmountBeforeDiscount,
       o.lineAmount,
       o.linePromo,
       o.itemQuantity,
       e.category,
       e.subcategory,
       CONCAT('Franchise_', TO_BASE64(SHA256(CAST(e.franchise AS STRING)))) AS franchise
FROM (
  SELECT *
  FROM offline_orders
  UNION ALL
  SELECT *
  FROM online_orders
) o
JOIN UniqueEANS e ON o.eanCode = e.eanCode
JOIN OrderNumberDB a ON o.OrderId = a.OrderId

WHERE o.CustomerId IS NOT NULL
AND o.OrderId IS NOT NULL
AND o.eanCode IS NOT NULL

ORDER BY CustomerId, OrderId, OrderDate DESC


```

## Email tracking

This Query analyzes tracking data for customer email interactions, including email opens, clicks, and opt-outs. Two Common Table Expressions (CTEs) are used

* First CTE, __top_device__, finds most common device used by each unique contact and date combination, based on number of email deliveries for that device.

* Second CTE, __tracking__, aggregates the tracking data by customer ID and delivery date, and calculates the number of email opens, clicks, and opt-outs.

The main query groups both and will be used to find device most often used for clicking and opening emails. For instance, it is possible that someones who always clicks email on a PC is more likely to convert as someone who clicks on an Iphone, as e-commerce conversion rates are lower on portable devices.

### Query

``` sql
-- CTE 1: Find the top device for each unique customer and date combination
WITH top_device AS (
SELECT distinct
    OCD_CONTACT_MASTER_ID as ocdMasterId, 
    log_date, 
    FIRST_VALUE(DEVICE) OVER (-- find the first (most common) device used by the customer on that date
      PARTITION BY OCD_CONTACT_MASTER_ID, log_date
      ORDER BY num_deliveries DESC -- order by number of deliveries (so we get the most common device first)
    ) AS Device
FROM (
  SELECT 
    OCD_CONTACT_MASTER_ID,
    DATE(TRACKINGLOG_DATE) as log_date, 
    DEVICE,
    COUNT(DISTINCT DELIVERY_ID_AC) as num_deliveries,
  FROM `emea-c1-dwh-prd.switzerland_all.tracking_log`
  WHERE REGEXP_CONTAINS(BRAND, brands)
  GROUP BY OCD_CONTACT_MASTER_ID, DATE(TRACKINGLOG_DATE), DEVICE
)
),

-- CTE 2: Aggregate tracking data by customer ID and delivery date
tracking AS (
 SELECT distinct
    OCD_CONTACT_MASTER_ID as ocdMasterId,
    DATE(TRACKINGLOG_DATE) as log_date,
    SUM(CASE WHEN TRACKINGLOG_URL_TYPE="Open" THEN 1 ELSE 0 END) AS OpenedEmail,
    SUM(CASE WHEN TRACKINGLOG_URL_TYPE="Email click" THEN 1 ELSE 0 END) AS ClickedEmail,
    SUM(CASE WHEN TRACKINGLOG_URL_TYPE="Opt-out" THEN 1 ELSE 0 END) AS OptoutEmail,
  FROM `emea-c1-dwh-prd.switzerland_all.tracking_log`
  WHERE REGEXP_CONTAINS(BRAND, brands)
  GROUP BY 1,2 ORDER BY 1)

-- Main query: join the two CTEs on customer ID and delivery date, and show the device used, email opens, clicks, and opt-outs
SELECT 
SHA256(top_device.ocdMasterId) AS CustomerID,
top_device.log_date, 
top_device.Device, 
tracking.OpenedEmail, 
tracking.ClickedEmail, 
tracking.OptoutEmail

FROM top_device
JOIN tracking ON top_device.ocdMasterId = tracking.ocdMasterId 
AND top_device.log_date = tracking.log_date

ORDER BY CustomerID
```